<a href="https://colab.research.google.com/github/hemingchun/clash-verge-rev/blob/main/%E5%B9%B6%E7%BD%91%E5%A4%9A%E8%83%BD%E4%BA%92%E8%A1%A5%E5%BE%AE%E7%94%B5%E7%BD%91%E7%BB%8F%E6%B5%8E%E8%B0%83%E5%BA%A6%E4%BC%98%E5%8C%96_(Pyomo).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 导入 Pyomo 库
from pyomo.environ import *
from pyomo.common.errors import ApplicationError # 导入 ApplicationError
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# --- 设置中文字体 ---
# 尝试使用 'SimHei' 字体，如果系统中没有，绘图时中文可能无法正常显示
# 您可以根据您的系统情况替换为其他可用的中文字体，例如 'Microsoft YaHei', 'FangSong' 等
try:
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 指定默认字体
    plt.rcParams['axes.unicode_minus'] = False    # 解决保存图像是负号'-'显示为方块的问题
except Exception as e:
    print(f"设置中文字体失败，可能需要安装SimHei字体或指定其他可用中文字体: {e}")
    # 可以尝试备选字体
    # try:
    #     plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
    #     plt.rcParams['axes.unicode_minus'] = False
    # except Exception as e2:
    #     print(f"设置备用中文字体Microsoft YaHei也失败: {e2}")


# 创建一个具体的优化模型
model = ConcreteModel()

# --- 1. 定义集合 ---
# 时间集合 (例如，一天24小时，每小时一个时间步)
model.T = RangeSet(1, 24)

# --- 2. 定义参数 ---
# 这些参数是说明性的，实际应用中需要替换为真实数据

# 电网参数
model.grid_price_purchase = Param(model.T, initialize={ # 从电网购电价格 (元/kWh)
    1: 0.4, 2: 0.4, 3: 0.4, 4: 0.4, 5: 0.4, 6: 0.4, 7: 0.4, 8: 1.2, 9: 1.2, 10: 1.2,
    11: 0.4, 12: 0.4, 13: 1.2, 14: 1.2, 15: 1.2, 16: 1.2, 17: 0.7, 18: 0.7, 19: 0.7,
    20: 0.7, 21: 0.7, 22: 0.7, 23: 0.7, 24: 0.7
})
model.grid_price_sell = Param(model.T, initialize=0.3) # 向电网售电价格 (元/kWh)
model.grid_max_power_purchase = Param(initialize=1000) # 从电网购电最大功率 (kW)
model.grid_max_power_sell = Param(initialize=800)   # 向电网售电最大功率 (kW)

# 光伏 (PV) 参数
model.pv_capacity = Param(initialize=500) # 光伏额定容量 (kW)
model.pv_generation_profile = Param(model.T, initialize={ # 光伏出力百分比 (相对于额定容量)
    1:0, 2:0, 3:0, 4:0, 5:0, 6:0.05, 7:0.1, 8:0.2, 9:0.4, 10:0.6,
    11:0.75, 12:0.85, 13:0.8, 14:0.7, 15:0.55, 16:0.35, 17:0.15, 18:0.05, 19:0,
    20:0, 21:0, 22:0, 23:0, 24:0
})

# 太阳能光热 (Solar Thermal) 参数 - 简化模型
model.st_capacity_heat = Param(initialize=300) # 太阳能光热额定产热功率 (kWth)
model.st_heat_profile = Param(model.T, initialize=model.pv_generation_profile) # 假设产热特性同光伏

# 小型燃气轮机 (Gas Turbine - GT) 参数
model.gt_min_power = Param(initialize=50)  # GT最小出力 (kW) - 当开启时
model.gt_max_power = Param(initialize=300) # GT最大出力 (kW)
model.gt_ramp_up_limit = Param(initialize=150) # GT向上爬坡速率 (kW/h)
model.gt_ramp_down_limit = Param(initialize=150) # GT向下爬坡速率 (kW/h)
model.gt_fuel_cost_coeff_a = Param(initialize=0.002) # 燃料成本系数 a (元/kW^2h) - 假设成本函数 C = aP^2 + bP + cU
model.gt_fuel_cost_coeff_b = Param(initialize=0.3)  # 燃料成本系数 b (元/kWh)
model.gt_fuel_cost_coeff_c = Param(initialize=10)   # 启动/固定成本 (元/h，当开启时)
model.gt_chp_heat_power_ratio = Param(initialize=0.8) # GT热电比 (产热功率/产电功率) - 简化CHP

# 燃料电池 (Fuel Cell - FC) 参数
model.fc_min_power = Param(initialize=20)   # FC最小出力 (kW) - 当开启时
model.fc_max_power = Param(initialize=150)  # FC最大出力 (kW)
model.fc_ramp_up_limit = Param(initialize=100) # FC向上爬坡速率 (kW/h)
model.fc_ramp_down_limit = Param(initialize=100) # FC向下爬坡速率 (kW/h)
model.fc_efficiency = Param(initialize=0.5) # FC发电效率
model.fc_fuel_price = Param(initialize=2.5)  # FC燃料价格 (元/kWh_fuel_input, 例如氢气等效价格)
model.fc_chp_heat_power_ratio = Param(initialize=0.6) # FC热电比 - 简化CHP
# model.fc_fixed_cost = Param(initialize=5) # 可选：为FC增加一个固定运行成本

# 储电系统 (Battery Energy Storage System - BESS) 参数
model.bess_capacity_energy = Param(initialize=500) # BESS额定容量 (kWh)
model.bess_capacity_power = Param(initialize=150)  # BESS额定充放电功率 (kW)
model.bess_soc_min = Param(initialize=0.2)         # BESS最小SOC
model.bess_soc_max = Param(initialize=0.9)         # BESS最大SOC
model.bess_soc_initial = Param(initialize=0.5)     # BESS初始SOC
model.bess_charge_efficiency = Param(initialize=0.95) # BESS充电效率
model.bess_discharge_efficiency = Param(initialize=0.95)# BESS放电效率

# 储热系统 (Thermal Energy Storage - TES) 参数 - 简化模型
model.tes_capacity_energy = Param(initialize=600) # TES额定容量 (kWh_th)
model.tes_capacity_power = Param(initialize=100)  # TES额定储放热功率 (kW_th)
model.tes_soh_min = Param(initialize=0.1)         # TES最小储热状态 (SOHeat)
model.tes_soh_max = Param(initialize=0.95)        # TES最大储热状态
model.tes_soh_initial = Param(initialize=0.6)     # TES初始储热状态
model.tes_charge_efficiency = Param(initialize=0.9) # TES储热效率
model.tes_discharge_efficiency = Param(initialize=0.9)# TES放热效率
model.tes_loss_coeff = Param(initialize=0.01)     # TES小时热损失系数

# 储冷系统 (Cold Energy Storage - CES) 参数 - 简化模型 (假设由电制冷机供冷)
model.ces_capacity_energy = Param(initialize=400) # CES额定容量 (kWh_cold)
model.ces_capacity_power = Param(initialize=80)   # CES额定储放冷功率 (kW_cold)
model.ces_socold_min = Param(initialize=0.1)      # CES最小储冷状态 (SOCold)
model.ces_socold_max = Param(initialize=0.95)     # CES最大储冷状态
model.ces_socold_initial = Param(initialize=0.6)  # CES初始储冷状态
model.ces_charge_efficiency = Param(initialize=0.9) # CES储冷效率 (制冷机效率已包含)
model.ces_discharge_efficiency = Param(initialize=0.9)# CES放冷效率
model.ces_loss_coeff = Param(initialize=0.01)      # CES小时冷损失系数
model.electric_chiller_cop = Param(initialize=3.0) # 电制冷机COP

# 动态建筑负荷参数
# 电负荷 (kWh) - 已按要求更新
model.load_electric_profile = Param(model.T, initialize={
    1:100, 2:90, 3:85, 4:80, 5:85, 6:100, 7:150, 8:200, 9:250, 10:280,
    11:300, 12:310, 13:300, 14:280, 15:270, 16:260, 17:280, 18:250, 19:250,
    20:200, 21:150, 22:150, 23:150, 24:120
})
# 热负荷 (kWh_th)
model.load_heat_profile = Param(model.T, initialize={
    1:50, 2:55, 3:60, 4:65, 5:70, 6:80, 7:70, 8:60, 9:50, 10:40,
    11:30, 12:25, 13:25, 14:30, 15:35, 16:40, 17:50, 18:60, 19:70,
    20:80, 21:75, 22:70, 23:60, 24:50
})
# 冷负荷 (kWh_cold)
model.load_cold_profile = Param(model.T, initialize={
    1:10, 2:10, 3:10, 4:10, 5:15, 6:20, 7:25, 8:30, 9:40, 10:50,
    11:60, 12:70, 13:80, 14:85, 15:80, 16:70, 17:60, 18:50, 19:40,
    20:30, 21:25, 22:20, 23:15, 24:10
})
# 柔性电负荷参数 (例如可调节的HVAC部分)
model.load_flexible_max_power = Param(model.T, initialize=50) # 每小时最大可调节功率 (kW)
model.load_flexible_total_energy_reduction = Param(initialize=100) # 一天总的可削减电量 (kWh)
model.comfort_penalty_coeff = Param(initialize=0.5) # 舒适度惩罚系数 (元/kWh_reduction) - 简化

# V2G 电动汽车集群参数 (简化为一个聚合体)
model.v2g_num_vehicles = Param(initialize=20) # 参与V2G的车辆数
model.v2g_battery_capacity_per_vehicle = Param(initialize=50) # 单车电池容量 (kWh)
model.v2g_power_per_vehicle = Param(initialize=7)    # 单车充放电功率 (kW)
model.v2g_soc_min = Param(initialize=0.2)
model.v2g_soc_max = Param(initialize=0.9)
model.v2g_soc_initial_avg = Param(initialize=0.6) # 平均初始SOC
model.v2g_charge_efficiency = Param(initialize=0.92)
model.v2g_discharge_efficiency = Param(initialize=0.92)
model.v2g_availability_profile = Param(model.T, initialize={ # V2G可用车辆比例 (影响充放电功率)
    1:0.1, 2:0.1, 3:0.1, 4:0.1, 5:0.1, 6:0.1, 7:0.2, 8:0.3, 9:0.8, 10:0.8,
    11:0.8, 12:0.8, 13:0.8, 14:0.8, 15:0.8, 16:0.8, 17:0.8, 18:0.5, 19:0.3,
    20:0.2, 21:0.1, 22:0.1, 23:0.1, 24:0.1
})
model.v2g_target_soc_at_departure = Param(initialize=0.8) # 整个V2G车队在离开时间点的平均目标SOC
model.v2g_departure_time_assumed = Param(initialize=18) # 假设V2G车辆主要在18点后陆续离开

# --- 3. 定义变量 ---
# 电网交互变量
model.P_grid_purchase = Var(model.T, within=NonNegativeReals) # 从电网购电功率
model.P_grid_sell = Var(model.T, within=NonNegativeReals)     # 向电网售电功率

# 分布式电源出力变量
model.P_pv = Var(model.T, within=NonNegativeReals) # 光伏实际出力
model.P_gt = Var(model.T, within=NonNegativeReals) # 燃气轮机出力
model.U_gt_on = Var(model.T, within=Binary)      # 燃气轮机启停状态 (1 for on, 0 for off)
model.P_fc = Var(model.T, within=NonNegativeReals) # 燃料电池出力
model.U_fc_on = Var(model.T, within=Binary)      # 燃料电池启停状态 (1 for on, 0 for off)
model.Q_st_useful = Var(model.T, within=NonNegativeReals) # 太阳能光热有效产热

# 储能系统变量
model.P_bess_charge = Var(model.T, within=NonNegativeReals)    # BESS充电功率
model.P_bess_discharge = Var(model.T, within=NonNegativeReals) # BESS放电功率
model.E_bess_soc = Var(model.T, within=NonNegativeReals)       # BESS储能量 (SOC*Capacity)

model.Q_tes_charge = Var(model.T, within=NonNegativeReals)     # TES储热功率
model.Q_tes_discharge = Var(model.T, within=NonNegativeReals)  # TES放热功率
model.E_tes_soh = Var(model.T, within=NonNegativeReals)        # TES储热量

# 储冷系统相关变量
model.Q_ces_charge = Var(model.T, within=NonNegativeReals) # 输入到储冷系统的冷功率 (kW_cold)
model.P_electric_chiller_input = Var(model.T, within=NonNegativeReals) # 电制冷机输入电功率
model.Q_ces_discharge = Var(model.T, within=NonNegativeReals)  # CES放冷功率
model.E_ces_socold = Var(model.T, within=NonNegativeReals)     # CES储冷量

# 灵活性负荷变量
model.P_load_flexible_reduction = Var(model.T, within=NonNegativeReals) # 柔性电负荷削减量

# V2G 变量
model.P_v2g_charge_total = Var(model.T, within=NonNegativeReals)    # V2G集群总充电功率
model.P_v2g_discharge_total = Var(model.T, within=NonNegativeReals) # V2G集群总放电功率
model.E_v2g_soc_total = Var(model.T, within=NonNegativeReals)       # V2G集群总储能量 (指整个车队的总能量)

# --- 4. 定义约束 ---

# 4.1 电功率平衡约束
def electric_power_balance_rule(m, t):
    return (m.P_grid_purchase[t] + m.P_pv[t] + m.P_gt[t] + m.P_fc[t] + m.P_bess_discharge[t] + m.P_v2g_discharge_total[t] ==
            m.P_grid_sell[t] + m.load_electric_profile[t] - m.P_load_flexible_reduction[t] +
            m.P_bess_charge[t] + m.P_v2g_charge_total[t] + m.P_electric_chiller_input[t])
model.electric_power_balance_constr = Constraint(model.T, rule=electric_power_balance_rule)

# 4.2 热功率平衡约束
def heat_power_balance_rule(m, t):
    gt_heat_production = m.P_gt[t] * m.gt_chp_heat_power_ratio
    fc_heat_production = m.P_fc[t] * m.fc_chp_heat_power_ratio
    return (m.Q_st_useful[t] + gt_heat_production + fc_heat_production + m.Q_tes_discharge[t] ==
            m.load_heat_profile[t] + m.Q_tes_charge[t])
model.heat_power_balance_constr = Constraint(model.T, rule=heat_power_balance_rule)

# 4.3 冷功率平衡约束
def cold_power_balance_rule(m, t):
    # 电制冷机产冷 + CES放冷 = 冷负荷 + 向CES储冷
    # Q_ces_charge 是指输入到储冷系统的冷功率
    return (m.P_electric_chiller_input[t] * m.electric_chiller_cop + m.Q_ces_discharge[t] ==
            m.load_cold_profile[t] + model.Q_ces_charge[t])
model.cold_power_balance_constr = Constraint(model.T, rule=cold_power_balance_rule)


# 4.4 电网交互约束
def grid_purchase_limit_rule(m, t):
    return m.P_grid_purchase[t] <= m.grid_max_power_purchase
model.grid_purchase_limit_constr = Constraint(model.T, rule=grid_purchase_limit_rule)

def grid_sell_limit_rule(m, t):
    return m.P_grid_sell[t] <= m.grid_max_power_sell
model.grid_sell_limit_constr = Constraint(model.T, rule=grid_sell_limit_rule)

# 4.5 分布式电源出力约束
def pv_output_rule(m, t):
    return m.P_pv[t] <= m.pv_capacity * m.pv_generation_profile[t]
model.pv_output_constr = Constraint(model.T, rule=pv_output_rule)
def pv_actual_output_lower_bound_rule(m, t): # 允许弃光
    return m.P_pv[t] >= 0
model.pv_actual_output_lower_bound_constr = Constraint(model.T, rule=pv_actual_output_lower_bound_rule)


def st_useful_heat_rule(m, t):
    return m.Q_st_useful[t] <= m.st_capacity_heat * m.st_heat_profile[t]
model.st_useful_heat_constr = Constraint(model.T, rule=st_useful_heat_rule)
def st_actual_output_lower_bound_rule(m,t): # 允许弃热
    return m.Q_st_useful[t] >=0
model.st_actual_output_lower_bound_constr = Constraint(model.T, rule=st_actual_output_lower_bound_rule)

# GT 功率约束 (使用二进制变量)
def gt_upper_power_limit_rule(m, t):
  return m.P_gt[t] <= m.gt_max_power * m.U_gt_on[t]
model.gt_upper_power_limit_constr = Constraint(model.T, rule=gt_upper_power_limit_rule)

def gt_lower_power_limit_rule(m, t):
  return m.P_gt[t] >= m.gt_min_power * m.U_gt_on[t]
model.gt_lower_power_limit_constr = Constraint(model.T, rule=gt_lower_power_limit_rule)

def gt_ramp_up_rule(m, t):
    if t == m.T.first():
        return Constraint.Skip
    return m.P_gt[t] - m.P_gt[t-1] <= m.gt_ramp_up_limit
model.gt_ramp_up_constr = Constraint(model.T, rule=gt_ramp_up_rule)

def gt_ramp_down_rule(m, t):
    if t == m.T.first():
        return Constraint.Skip
    return m.P_gt[t-1] - m.P_gt[t] <= m.gt_ramp_down_limit
model.gt_ramp_down_constr = Constraint(model.T, rule=gt_ramp_down_rule)

# FC 功率约束 (使用二进制变量)
def fc_upper_power_limit_rule(m, t):
  return m.P_fc[t] <= m.fc_max_power * m.U_fc_on[t]
model.fc_upper_power_limit_constr = Constraint(model.T, rule=fc_upper_power_limit_rule)

def fc_lower_power_limit_rule(m, t):
  return m.P_fc[t] >= m.fc_min_power * m.U_fc_on[t]
model.fc_lower_power_limit_constr = Constraint(model.T, rule=fc_lower_power_limit_rule)

def fc_ramp_up_rule(m, t):
    if t == m.T.first():
        return Constraint.Skip
    return m.P_fc[t] - m.P_fc[t-1] <= m.fc_ramp_up_limit
model.fc_ramp_up_constr = Constraint(model.T, rule=fc_ramp_up_rule)

def fc_ramp_down_rule(m, t):
    if t == m.T.first():
        return Constraint.Skip
    return m.P_fc[t-1] - m.P_fc[t] <= m.fc_ramp_down_limit
model.fc_ramp_down_constr = Constraint(model.T, rule=fc_ramp_down_rule)

# 4.6 储电系统 (BESS) 约束
def bess_soc_rule(m, t):
    if t == m.T.first():
        return m.E_bess_soc[t] == m.bess_soc_initial * m.bess_capacity_energy + \
               m.P_bess_charge[t] * m.bess_charge_efficiency - m.P_bess_discharge[t] / m.bess_discharge_efficiency
    return m.E_bess_soc[t] == m.E_bess_soc[t-1] + \
           m.P_bess_charge[t] * m.bess_charge_efficiency - m.P_bess_discharge[t] / m.bess_discharge_efficiency
model.bess_soc_constr = Constraint(model.T, rule=bess_soc_rule)

def bess_soc_limits_rule(m, t):
    return inequality(m.bess_soc_min * m.bess_capacity_energy, m.E_bess_soc[t], m.bess_soc_max * m.bess_capacity_energy)
model.bess_soc_limits_constr = Constraint(model.T, rule=bess_soc_limits_rule)

def bess_charge_power_limit_rule(m, t):
    return m.P_bess_charge[t] <= m.bess_capacity_power
model.bess_charge_power_limit_constr = Constraint(model.T, rule=bess_charge_power_limit_rule)

def bess_discharge_power_limit_rule(m, t):
    return m.P_bess_discharge[t] <= m.bess_capacity_power
model.bess_discharge_power_limit_constr = Constraint(model.T, rule=bess_discharge_power_limit_rule)

# *** 新增约束 ***: 确保BESS在一天结束时SOC恢复到初始值
def bess_soc_final_rule(m):
    return m.E_bess_soc[m.T.last()] == m.bess_soc_initial * m.bess_capacity_energy
model.bess_soc_final_constr = Constraint(rule=bess_soc_final_rule)

# 4.7 储热系统 (TES) 约束
def tes_soh_rule(m, t):
    if t == m.T.first():
        return m.E_tes_soh[t] == m.tes_soh_initial * m.tes_capacity_energy * (1 - m.tes_loss_coeff) + \
               m.Q_tes_charge[t] * m.tes_charge_efficiency - m.Q_tes_discharge[t] / m.tes_discharge_efficiency
    return m.E_tes_soh[t] == m.E_tes_soh[t-1] * (1 - m.tes_loss_coeff) + \
           m.Q_tes_charge[t] * m.tes_charge_efficiency - m.Q_tes_discharge[t] / m.tes_discharge_efficiency
model.tes_soh_constr = Constraint(model.T, rule=tes_soh_rule)

def tes_soh_limits_rule(m, t):
    return inequality(m.tes_soh_min * m.tes_capacity_energy, m.E_tes_soh[t], m.tes_soh_max * m.tes_capacity_energy)
model.tes_soh_limits_constr = Constraint(model.T, rule=tes_soh_limits_rule)

def tes_charge_power_limit_rule(m, t):
    return m.Q_tes_charge[t] <= m.tes_capacity_power
model.tes_charge_power_limit_constr = Constraint(model.T, rule=tes_charge_power_limit_rule)

def tes_discharge_power_limit_rule(m, t):
    return m.Q_tes_discharge[t] <= m.tes_capacity_power
model.tes_discharge_power_limit_constr = Constraint(model.T, rule=tes_discharge_power_limit_rule)

# *** 新增约束 ***: 确保TES在一天结束时储热状态恢复到初始值
def tes_soh_final_rule(m):
    return m.E_tes_soh[m.T.last()] == m.tes_soh_initial * m.tes_capacity_energy
model.tes_soh_final_constr = Constraint(rule=tes_soh_final_rule)

# 4.8 储冷系统 (CES) 约束
# Q_ces_charge 是输入到储冷系统的冷功率
def ces_socold_rule(m, t):
    if t == m.T.first():
        return m.E_ces_socold[t] == m.ces_socold_initial * m.ces_capacity_energy * (1 - m.ces_loss_coeff) + \
               model.Q_ces_charge[t] * m.ces_charge_efficiency - \
               m.Q_ces_discharge[t] / m.ces_discharge_efficiency
    return m.E_ces_socold[t] == m.E_ces_socold[t-1] * (1 - m.ces_loss_coeff) + \
           model.Q_ces_charge[t] * m.ces_charge_efficiency - \
           m.Q_ces_discharge[t] / m.ces_discharge_efficiency
model.ces_socold_constr = Constraint(model.T, rule=ces_socold_rule)


def ces_socold_limits_rule(m, t):
    return inequality(m.ces_socold_min * m.ces_capacity_energy, m.E_ces_socold[t], m.ces_socold_max * m.ces_capacity_energy)
model.ces_socold_limits_constr = Constraint(model.T, rule=ces_socold_limits_rule)

def ces_charge_power_limit_rule(m, t): # Q_ces_charge 是输入储冷罐的冷功率
    return model.Q_ces_charge[t] <= m.ces_capacity_power
model.ces_charge_power_limit_constr = Constraint(model.T, rule=ces_charge_power_limit_rule)

def ces_discharge_power_limit_rule(m, t):
    return m.Q_ces_discharge[t] <= m.ces_capacity_power
model.ces_discharge_power_limit_constr = Constraint(model.T, rule=ces_discharge_power_limit_rule)

# *** 新增约束 ***: 确保CES在一天结束时储冷状态恢复到初始值
def ces_socold_final_rule(m):
    return m.E_ces_socold[m.T.last()] == m.ces_socold_initial * m.ces_capacity_energy
model.ces_socold_final_constr = Constraint(rule=ces_socold_final_rule)

# 4.9 柔性电负荷约束
def flexible_load_reduction_limit_rule(m, t):
    return m.P_load_flexible_reduction[t] <= m.load_flexible_max_power[t]
model.flexible_load_reduction_limit_constr = Constraint(model.T, rule=flexible_load_reduction_limit_rule)

def flexible_load_total_energy_reduction_rule(m):
    return sum(m.P_load_flexible_reduction[t] for t in m.T) <= m.load_flexible_total_energy_reduction
model.flexible_load_total_energy_reduction_constr = Constraint(rule=flexible_load_total_energy_reduction_rule)

# 4.10 V2G 集群约束
# E_v2g_soc_total 代表整个车队的总储能量
def v2g_soc_total_rule(m, t):
    if t == m.T.first():
        # 初始总能量 = 平均初始SOC * 总车辆数 * 单车容量
        initial_total_fleet_energy = m.v2g_soc_initial_avg * m.v2g_num_vehicles * m.v2g_battery_capacity_per_vehicle
        return m.E_v2g_soc_total[t] == initial_total_fleet_energy + \
               m.P_v2g_charge_total[t] * m.v2g_charge_efficiency - m.P_v2g_discharge_total[t] / m.v2g_discharge_efficiency
    return m.E_v2g_soc_total[t] == m.E_v2g_soc_total[t-1] + \
           m.P_v2g_charge_total[t] * m.v2g_charge_efficiency - m.P_v2g_discharge_total[t] / m.v2g_discharge_efficiency
model.v2g_soc_total_constr = Constraint(model.T, rule=v2g_soc_total_rule)

def v2g_soc_total_limits_rule(m, t):
    # 总储能量的上下限应基于整个车队的总容量
    max_total_fleet_energy = m.v2g_soc_max * m.v2g_num_vehicles * m.v2g_battery_capacity_per_vehicle
    min_total_fleet_energy = m.v2g_soc_min * m.v2g_num_vehicles * m.v2g_battery_capacity_per_vehicle
    return inequality(min_total_fleet_energy, m.E_v2g_soc_total[t], max_total_fleet_energy)
model.v2g_soc_total_limits_constr = Constraint(model.T, rule=v2g_soc_total_limits_rule)


def v2g_charge_total_power_limit_rule(m, t):
    # 总充电功率受限于当前可用车辆的总充电能力
    return m.P_v2g_charge_total[t] <= m.v2g_num_vehicles * m.v2g_power_per_vehicle * m.v2g_availability_profile[t]
model.v2g_charge_total_power_limit_constr = Constraint(model.T, rule=v2g_charge_total_power_limit_rule)

def v2g_discharge_total_power_limit_rule(m, t):
    # 总放电功率受限于当前可用车辆的总放电能力
    return m.P_v2g_discharge_total[t] <= m.v2g_num_vehicles * m.v2g_power_per_vehicle * m.v2g_availability_profile[t]
model.v2g_discharge_total_power_limit_constr = Constraint(model.T, rule=v2g_discharge_total_power_limit_rule)

def v2g_final_soc_target_rule(m):
    # 目标是整个车队的平均SOC达到预设值
    target_total_fleet_energy_at_departure = m.v2g_target_soc_at_departure * \
                                             m.v2g_num_vehicles * \
                                             m.v2g_battery_capacity_per_vehicle
    if m.v2g_departure_time_assumed in m.T:
         return m.E_v2g_soc_total[m.v2g_departure_time_assumed] >= target_total_fleet_energy_at_departure
    # 如果假设的离开时间不在T的范围内 (例如 T=RangeSet(1,17) 而 departure_time_assumed=18),
    # 这个约束可能需要调整，例如应用到最后一个时间步，或者确保departure_time_assumed总是在T内。
    # 当前的实现是如果不在T内则跳过，这可能不是期望的行为，取决于具体需求。
    return Constraint.Skip
model.v2g_final_soc_target_constr = Constraint(rule=v2g_final_soc_target_rule)

# *** 新增约束 ***: 确保V2G集群在一天结束时总储能量恢复到初始值
def v2g_soc_total_final_rule(m):
    initial_total_fleet_energy = m.v2g_soc_initial_avg * m.v2g_num_vehicles * m.v2g_battery_capacity_per_vehicle
    return m.E_v2g_soc_total[m.T.last()] == initial_total_fleet_energy
model.v2g_soc_total_final_constr = Constraint(rule=v2g_soc_total_final_rule)


# --- 5. 定义目标函数 ---
def objective_rule(m):
    cost_grid_purchase = sum(m.P_grid_purchase[t] * m.grid_price_purchase[t] for t in m.T)
    revenue_grid_sell = sum(m.P_grid_sell[t] * m.grid_price_sell[t] for t in m.T)

    cost_gt_fuel = sum(m.gt_fuel_cost_coeff_a * m.P_gt[t]**2 + \
                       m.gt_fuel_cost_coeff_b * m.P_gt[t] + \
                       m.gt_fuel_cost_coeff_c * m.U_gt_on[t]
                       for t in m.T)
    cost_fc_fuel = sum((m.P_fc[t] / m.fc_efficiency) * m.fc_fuel_price for t in m.T)


    cost_flexible_load_penalty = sum(m.P_load_flexible_reduction[t] * m.comfort_penalty_coeff for t in m.T)

    return cost_grid_purchase + cost_gt_fuel + cost_fc_fuel + cost_flexible_load_penalty - revenue_grid_sell
model.objective = Objective(rule=objective_rule, sense=minimize)

# --- 6. 求解模型 ---
try:
    solver_name = 'cplex' # 指定使用 CPLEX
    # 请将下面的路径替换为您CPLEX的实际安装路径下的cplex.exe文件路径
    # 例如: 'C:/Program Files/IBM/ILOG/CPLEX_Studio1210/cplex/bin/x64_win64/cplex.exe'
    # 如果CPLEX可执行文件已经在系统PATH中，则不需要指定executable
    cplex_executable_path = 'F:/software/IBM/ILOG/CPLEX_Enterprise_Server1210/CPLEX_Studio/cplex/bin/x64_win64/cplex.exe'

    # 检查路径是否正确，如果CPLEX在系统PATH中，可以尝试不带executable参数
    try:
        solver = SolverFactory(solver_name, executable=cplex_executable_path)
    except ApplicationError: # 已在顶部导入
        print(f"错误：无法在指定路径找到CPLEX可执行文件: {cplex_executable_path}")
        print("如果CPLEX已添加到系统PATH，请尝试不使用 'executable' 参数调用 SolverFactory('cplex')")
        solver = SolverFactory(solver_name) # 尝试不带路径，看是否在PATH中

    results = solver.solve(model, tee=True)

    # --- 7. 结果分析与可视化 (简化) ---
    if (results.solver.status == SolverStatus.ok) and \
       (results.solver.termination_condition == TerminationCondition.optimal):
        print("\n--- 优化成功 (使用 CPLEX) ---")
        print(f"最小总运行成本: {model.objective():.2f} 元")

        time_steps = list(model.T)
        P_grid_purchase_res = [value(model.P_grid_purchase[t]) for t in model.T]
        P_grid_sell_res = [value(model.P_grid_sell[t]) for t in model.T]
        P_pv_res = [value(model.P_pv[t]) for t in model.T]
        P_gt_res = [value(model.P_gt[t]) for t in model.T]
        U_gt_on_res = [value(model.U_gt_on[t]) for t in model.T]
        P_fc_res = [value(model.P_fc[t]) for t in model.T]
        U_fc_on_res = [value(model.U_fc_on[t]) for t in model.T]
        P_bess_charge_res = [value(model.P_bess_charge[t]) for t in model.T]
        P_bess_discharge_res = [value(model.P_bess_discharge[t]) for t in model.T]
        E_bess_soc_res = [value(model.E_bess_soc[t]) / model.bess_capacity_energy for t in model.T]
        P_load_total_electric = [value(model.load_electric_profile[t] - model.P_load_flexible_reduction[t]) for t in model.T]
        P_flex_reduct_res = [value(model.P_load_flexible_reduction[t]) for t in model.T]

        P_v2g_charge_res = [value(model.P_v2g_charge_total[t]) for t in model.T]
        P_v2g_discharge_res = [value(model.P_v2g_discharge_total[t]) for t in model.T]
        # 计算V2G SOC百分比时，分母应为整个车队的总容量
        v2g_total_fleet_capacity = model.v2g_num_vehicles * model.v2g_battery_capacity_per_vehicle
        E_v2g_soc_total_res = [value(model.E_v2g_soc_total[t]) / (v2g_total_fleet_capacity + 1e-6) for t in model.T]


        Q_st_res = [value(model.Q_st_useful[t]) for t in model.T]
        Q_tes_charge_res = [value(model.Q_tes_charge[t]) for t in model.T]
        Q_tes_discharge_res = [value(model.Q_tes_discharge[t]) for t in model.T]
        E_tes_soh_res = [value(model.E_tes_soh[t]) / model.tes_capacity_energy for t in model.T]

        P_chiller_in_res = [value(model.P_electric_chiller_input[t]) for t in model.T]
        Q_ces_charge_res = [value(model.Q_ces_charge[t]) for t in model.T]
        Q_ces_discharge_res = [value(model.Q_ces_discharge[t]) for t in model.T]
        E_ces_socold_res = [value(model.E_ces_socold[t]) / model.ces_capacity_energy for t in model.T]


        print(f"\nGT On/Off: {U_gt_on_res}")
        print(f"FC On/Off: {U_fc_on_res}")

        # 图 1: 电网交互与净负荷
        plt.figure(figsize=(10, 6))
        plt.plot(time_steps, P_grid_purchase_res, label='电网购电 (kW)', marker='o', linestyle='-')
        plt.plot(time_steps, P_grid_sell_res, label='向电网售电 (kW)', marker='x', linestyle='-')
        plt.plot(time_steps, P_load_total_electric, label='净电负荷 (kW)', linestyle='--', color='black')
        plt.ylabel('功率 (kW)')
        plt.xlabel('时间 (h)')
        plt.title('电网交互与净电负荷')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # 图 2: 分布式电源发电
        plt.figure(figsize=(10, 6))
        plt.plot(time_steps, P_pv_res, label='光伏出力 (kW)', marker='s', color='orange')
        plt.plot(time_steps, P_gt_res, label='燃气轮机出力 (kW)', marker='^', color='brown')
        plt.plot(time_steps, P_fc_res, label='燃料电池出力 (kW)', marker='v', color='purple')
        plt.ylabel('功率 (kW)')
        plt.xlabel('时间 (h)')
        plt.title('分布式电源发电情况')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # 图 3: 储电系统功率
        plt.figure(figsize=(10, 6))
        plt.plot(time_steps, P_bess_charge_res, label='储电系统充电功率 (kW)', marker='>', color='green')
        plt.plot(time_steps, P_bess_discharge_res, label='储电系统放电功率 (kW)', marker='<', color='red')
        plt.ylabel('功率 (kW)')
        plt.xlabel('时间 (h)')
        plt.title('储电系统充放电功率')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # 图 4: 储电系统SOC
        plt.figure(figsize=(10, 6))
        plt.plot(time_steps, E_bess_soc_res, label='储电系统SOC', marker='.', color='blue')
        plt.ylabel('SOC')
        plt.xlabel('时间 (h)')
        plt.ylim(0, 1)
        plt.title('储电系统SOC变化')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # 图 5: V2G聚合功率
        plt.figure(figsize=(10, 6))
        plt.plot(time_steps, P_v2g_charge_res, label='V2G充电总功率 (kW)', marker='>', color='cyan')
        plt.plot(time_steps, P_v2g_discharge_res, label='V2G放电总功率 (kW)', marker='<', color='magenta')
        plt.ylabel('功率 (kW)')
        plt.xlabel('时间 (h)')
        plt.title('V2G聚合充放电功率')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # 图 6: V2G聚合SOC
        plt.figure(figsize=(10, 6))
        plt.plot(time_steps, E_v2g_soc_total_res, label='V2G平均SOC', marker='.', color='lime')
        plt.ylabel('SOC')
        plt.xlabel('时间 (h)')
        plt.ylim(0,1)
        plt.title('V2G聚合SOC变化')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # 图 7: 热平衡组件
        plt.figure(figsize=(12, 7)) # 稍微调整尺寸以容纳更多图例
        heat_load_values = list(model.load_heat_profile.values())
        plt.plot(time_steps, heat_load_values, label='热负荷 (kWth)', linestyle='--', color='red')
        plt.plot(time_steps, Q_st_res, label='太阳能光热产热 (kWth)', marker='p', color='darkorange')
        gt_heat_prod = [value(model.P_gt[t] * model.gt_chp_heat_power_ratio) for t in model.T]
        fc_heat_prod = [value(model.P_fc[t] * model.fc_chp_heat_power_ratio) for t in model.T]
        plt.plot(time_steps, gt_heat_prod, label='燃气轮机产热 (kWth)', marker='h', color='saddlebrown')
        plt.plot(time_steps, fc_heat_prod, label='燃料电池产热 (kWth)', marker='H', color='indigo')
        plt.plot(time_steps, Q_tes_discharge_res, label='储热系统放热 (kWth)', marker='<', color='salmon')
        plt.plot(time_steps, Q_tes_charge_res, label='储热系统储热 (kWth)', marker='>', color='lightcoral', linestyle=':')
        plt.ylabel('热功率 (kWth)')
        plt.xlabel('时间 (h)')
        plt.title('热平衡各组件功率')
        plt.legend(fontsize='small', loc='best')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # 图 8: 冷平衡组件
        plt.figure(figsize=(12, 7)) # 稍微调整尺寸以容纳更多图例
        cold_load_values = list(model.load_cold_profile.values())
        plt.plot(time_steps, cold_load_values, label='冷负荷 (kWc)', linestyle='--', color='blue')
        chiller_cold_out = [value(model.P_electric_chiller_input[t] * model.electric_chiller_cop) for t in model.T]
        plt.plot(time_steps, chiller_cold_out, label='电制冷机产冷 (kWc)', marker='d', color='deepskyblue')
        plt.plot(time_steps, Q_ces_discharge_res, label='储冷系统放冷 (kWc)', marker='<', color='aqua')
        plt.plot(time_steps, Q_ces_charge_res, label='储冷系统储冷 (kWc)', marker='>', color='powderblue', linestyle=':') # Q_ces_charge 是输入储冷的功率
        plt.ylabel('冷功率 (kWc)')
        plt.xlabel('时间 (h)')
        plt.title('冷平衡各组件功率')
        plt.legend(fontsize='small', loc='best')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    elif results.solver.termination_condition == TerminationCondition.infeasible:
        print("--- 模型不可行 (使用 CPLEX) ---")
        print("请检查约束条件是否存在冲突。")
        # from pyomo.util.infeasible import log_infeasible_constraints
        # log_infeasible_constraints(model, log_expression=True, log_variables=True)
        # print("\nVariable bounds:")
        # for v_name in model.component_map(Var, active=True):
        #     v = getattr(model, v_name)
        #     for index in v:
        #         print(f"{v[index].name}: lb={v[index].lb}, ub={v[index].ub}, value={value(v[index], exception=False)}")

    else:
        print(f"--- CPLEX 求解器状态: {results.solver.status} ---")
        print(f"--- CPLEX 终止条件: {results.solver.termination_condition} ---")

except ApplicationError as app_err: # 已在顶部导入
    print(f"Pyomo 应用错误 (通常是求解器未找到或配置问题): {app_err}")
    print("请确保CPLEX已正确安装，并且Pyomo可以找到其可执行文件。")
    print(f"尝试的CPLEX路径是: {cplex_executable_path}")
    print("如果CPLEX在系统PATH中，有时不需要显式指定路径。")
except Exception as e:
    print(f"求解过程中发生其他错误: {e}")